In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key="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",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

response=llm.invoke("The first person to land on moon was..")
print(response.content)

Neil Armstrong.  He became the first person to set foot on the Moon on July 20, 1969, as part of NASA's Apollo 11 mission.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results?q=%22Software%20Engineer%22&location=India&src=Online/House%20Ads/BKWS_LOC2&gad_source=1&gclid=CjwKCAiAudG5BhAREiwAWMlSjAimwKFXM1qfcHWBpJPhMiWwlSKZAssGjAi3jMSN0VBHqXlXGumxgxoC7SgQAvD_BwE")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Search Jobs — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend FeedbackSearch JobsSearch sidebar149  jobs matchedClear filtersWhat do you want to do?Locationsexpand_lessIndiaWhich location(s) do you prefer?search1 / 4LocationsRemote eligibleinfo_outlineThe remote filter will show you jobs that are eligible for remote work. Some roles allow for remote work and/or in-office work.Candidates will be able to share location preferences later in the hiring process.Experienceexpand_moreSkills & qualificationsexpand_moreDegreeexpand_moreJob typesexpand_moreOrganizationsexpand_moreSort byexpand_moreFollow Life at Google on More about usAbout usopen_in_newContact usopen_in_newPressopen_in_newRelated InformationInvestor relationsopen_in_newBlogopen_in_newEqual OpportunityGoogle is prou

In [41]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

[
    {
        "role": "Senior Software Engineer, Scaling On-Device ML Frameworks",
        "experience": "Mid",
        "skills": ["Software development in one or more programming languages", "Data structures/algorithms", "Speech/audio", "Reinforcement learning", "ML infrastructure"],
        "description": "Experience driving progress, solving problems, and mentoring more junior team members; deeper expertise and applied knowledge within relevant area."
    },
    {
        "role": "Software Engineer, YouTube",
        "experience": "Early",
        "skills": ["Software development in one or more programming languages"],
        "description": "Experience completing work as directed, and collaborating with teammates; developing knowledge of relevant concepts and processes."
    },
    {
        "role": "Software Engineering Manager, Corporate Engineering",
        "experience": "Advanced",
        "skills": ["Software development in one or more programming languages", "Technical lea

str

In [28]:
from langchain_core.output_parsers import JsonOutputParser

json_parser= JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res
type(json_res)

list

In [6]:
import pandas as pd
df=pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
import uuid
import chromadb
chroma_client = chromadb.PersistentClient('vectorstore')
collection = chroma_client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links":row["Links"]},
                      ids=[str(uuid.uuid4())])

In [33]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [44]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rahul, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Efficiency and Scalability with AtliQ's AI & Software Solutions

Dear [Recipient's Name],

I came across your organization and was impressed by the innovative work you're doing in [industry/field]. As a Business Development Executive at AtliQ, I'd like to introduce you to our expertise in leveraging AI and software consulting to optimize business processes.

At AtliQ, we empower enterprises to achieve scalability, process optimization, cost reduction, and enhanced overall efficiency through customized solutions. Our team of experts is well-versed in developing automated tools to streamline operations, drive growth, and improve bottom-line results.

Some of our notable solutions include:

* Machine Learning and Python-based solutions that drive business value (check out our portfolio: https://example.com/ml-python-portfolio)
* Custom Python solutions that cater to specific business needs (explore our Python portfolio: https://example.com/python-portfolio)

Our capabiliti